# Sistema de Modelado y Análisis de Supervivencia (médica)

En este notebook se detalla un sistema o secuencia de pasos a seguir que se pueden aplicar a cualquier modelo de Machine Learning que involucren problemas de tipo ***supervivencia***.

## 1. Dataset
* Cada fila representa a un paciente.
* El objetivo es predecir el riesgo (**risk**) de que un paciente muera
    * Esto se determina en la columna **event**.
---
En los análisis de supervivencia tenemos:
* Los targets:
    * **event**: True/False (muere/no muere).
    * **time**: El tiempo cuando el evento ocurre.
* Explanatory variables:
    * El resto de variables (age, prior_therapy)

Por lo cual la formula de la ecuación matemática del modelo será:
$$
risk = (w_0) + (w_1) \cdot age + (w_2) \cdot prior\_therapy
$$

Con los modelos de Machine Learning, se busca encontrar los mejores valores (optimización) para los pesos ***w1*** y ***w2*** de la ecuación anterior, para finalmente calcular el **risk** asociado.

In [5]:
import pandas as pd

df_patients = pd.read_excel("../data/data_lung_cancer_smote.xlsx").loc[:,["event", "time", "age", "prior_therapy"]].assign(prior_therapy=lambda df: df.prior_therapy.replace({"Yes": 1, "No": 0}))
df_patients

,event,time,age,prior_therapy
0,True,2.373626,69.000000,0
1,True,7.516484,38.000000,0
2,True,4.153846,63.000000,1
3,True,3.890110,65.000000,1
4,True,0.329670,49.000000,0
...,...,...,...,...
238,False,3.142881,65.810046,0
239,False,3.380047,36.495508,1
240,False,3.082424,65.029553,0
241,False,2.986648,62.424988,0


## 2. Feature Selection

Selección de las variables a utilizar en el modelo:
* `y (target)`: **event** y **time**
    * Para que estas variables puedan ser procesadas por el modelo, deben transformarse a otra estructura de datos (*numpy records array*). Esto se hace con *.to_records()*
* `x (explanatory)`: Variables relevantes para calcular el riesgo de un paciente.

In [11]:
y = df_patients[["event", "time"]].to_records(index=False)

In [13]:
X = df_patients.drop(["event", "time"], axis=1)

## 3. The Cox PH Model

### 3.1 Fit - Ajuste del modelo (ecuación matemática)

In [16]:
from sksurv.linear_model import CoxPHSurvivalAnalysis

In [17]:
model_cox = CoxPHSurvivalAnalysis()

In [18]:
model_cox.fit(X, y)     # CORREGIR A FUTURO: Se está utilizando el dataset completo para entrenar, lo cual es un error. Se debe crear Train y Test sets.

CoxPHSurvivalAnalysis()

In [19]:
model_cox.__dict__

{'alpha': 0,
 'ties': 'breslow',
 'n_iter': 100,
 'tol': 1e-09,
 'verbose': 0,
 '_baseline_model': <sksurv.linear_model.coxph.BreslowEstimator at 0x2824728f910>,
 'feature_names_in_': array(['age', 'prior_therapy'], dtype=object),
 'n_features_in_': 2,
 'coef_': array([ 0.02027216, -0.03710247])}

### 3.2 Predict - Predicciones con el modelo ajustado

In [20]:
model_cox.predict(X)

array([1.39877927, 0.77034221, 1.24004382, 1.28058815, 0.99333601,
       1.3616768 , 1.37850711, 0.87170303, 1.41905144, 1.64204524,
       1.24004382, 1.27714629, 1.01705002, 0.93596137, 1.23660197,
       0.70952572, 1.24004382, 1.13524115, 1.11496899, 1.35823495,
       1.24004382, 1.31769062, 0.93251952, 1.03732219, 1.39877927,
       1.37850711, 0.83460055, 1.07786651, 0.85143086, 1.29741846,
       1.31769062, 1.31769062, 1.11496899, 1.30086031, 1.2163298 ,
       1.35823495, 1.07442466, 1.25687413, 1.35823495, 1.45959576,
       0.97306384, 1.32113248, 1.1994995 , 1.2163298 , 1.25687413,
       0.77034221, 1.01360817, 1.24004382, 1.29741846, 0.87170303,
       0.68925356, 1.30086031, 1.25687413, 1.0541525 , 1.27714629,
       1.34140464, 0.91224735, 0.79405623, 1.33796278, 1.25687413,
       1.2163298 , 1.33796278, 0.77034221, 1.03732219, 0.71296757,
       1.2163298 , 0.97306384, 1.01705002, 1.41905144, 1.25687413,
       1.28058815, 1.29741846, 1.27714629, 1.01705002, 0.70952

### 3.3 Score - Evaluación de las predicciones

In [21]:
model_cox.score(X, y)

0.5637832080941956

El modelo tiene un rendimiento bastante pobre, solo un 56% de las predicciones son correctas. Con este resultado es incluso casi igual que lanzar una moneda al aire y determinar si el paciente muere o no, ya que las chances del lanzmaiento de la moneda son 50% (true) y 50%(false).

Se crearán nuevos modelos para comparar si alguno de ellos obtiene un mejor Score.  
***No olvidar que el modelo fue entrenado con el 100% de los datos, lo cual es un error. En futuros fix se dividirá en train y test sets.***